In [ ]:
from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_periodogram, seasonal_plot
from scipy.signal import periodogram
import seaborn as sns
from sklearn.linear_model import LinearRegression
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import matplotlib.pyplot as plt
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
whole = pd.read_csv(('/kaggle/input/petal3/petal.csv'), parse_dates=["Date"])
# add train and test files later

print(whole['Date'].dtypes)# confirm the dates are set up as datetime

**Begin Data Exploration**
Take a look at the data, columns names, data types.
Start by making a periodogram to see if there are seasonal and other features with periodic frequency to the time series.

In [ ]:
whole.head()

In [ ]:
whole.rename(columns={'Est. Gross Profit (Line)': 'Sales'}, inplace=True)
whole.head()

In [ ]:
print(len(whole))
whole.isna().sum()

In [ ]:
whole_sales=whole.groupby('Date')['Sales'].sum() 

def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("365D") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots(figsize=(12, 5))
     
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        ["Annual (1)","Semiannual (2)","Quarterly (4)","Bimonthly (6)","Monthly (12)","Biweekly (26)","Weekly (52)","Semiweekly (104)",],rotation=30,)
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

plot_periodogram(whole_sales)

The periodogram looks unusual. The annual peaks very high and wide and other peaks are jagged and indistinct. This could be due to inconsisent sales over time.

In [ ]:

whole_sales = whole_sales.reset_index()
whole_sales.columns = ['Date', 'Sales']

whole_sales['Date'] = pd.to_datetime(whole_sales['Date'])


whole_sales.sort_values('Date', inplace=True)


plt.figure(figsize=(10, 5))  # Set the size of the plot
plt.plot(whole_sales['Date'], whole_sales['Sales'], marker='o')  # Plot with markers


plt.title('Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')


plt.gcf().autofmt_xdate()

plt.show()



Here we can see that sales from mid-2016-2017 are much higher than the rest of the dataset. These high sales in 2016 will not be helful in predicting the sale behavior in the rest of the dataset. Additionally, there is an evident gap in sales in 2017. Take a closer look at the sales over time starting in 2018.

In [ ]:
#find the end date for the dataset
whole_sales.tail()

In [ ]:
#limit the whole_sales dataset to dates starting in 2018 
whole_sales.set_index('Date', inplace=True)
whole_sales.sort_index(inplace=True)

# Slice the DataFrame to the desired date range
truncated_whole_sales = whole_sales.loc['2018-01-01':'2020-08-10']
#reset and resort the index for graphing 
truncated_whole_sales = truncated_whole_sales.reset_index()
whole_sales.sort_values('Date', inplace=True)

plt.figure(figsize=(10, 5)) 
plt.plot(truncated_whole_sales['Date'], truncated_whole_sales['Sales'], marker='o')  # Plot with markers


plt.title('Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')

plt.gcf().autofmt_xdate()

plt.show()

This looks much better. It would be ideal to close in on the sections where there are no gaps. Closing into mid-April 2018 and ending with the expected sales pause on March 17, 2020.

In [ ]:
truncated_whole_sales = whole_sales.loc['2018-04-15':'2020-03-17']
#reset and resort the index for graphing 
truncated_whole_sales = truncated_whole_sales.reset_index()
#whole_sales.sort_values('Date', inplace=True)

plt.figure(figsize=(10, 5))  # Set the size of the plot
plt.plot(truncated_whole_sales['Date'], truncated_whole_sales['Sales'], marker='o')  # Plot with markers

plt.title('Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')

plt.gcf().autofmt_xdate()

plt.show()

In [ ]:
truncated_whole_sales.head(10) # starting sales with 04-26-2018

In [ ]:
truncated_whole_sales = whole_sales.loc['2018-04-15':'2020-03-17']
#reset and resort the index for graphing 
truncated_whole_sales = truncated_whole_sales.reset_index()

truncated_whole_sales.head()

In [ ]:
#Revisiting Periodogram
truncated_whole_sales=truncated_whole_sales.groupby('Date')['Sales'].sum()

def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("365D") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots(figsize=(12, 5))
     
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        ["Annual (1)","Semiannual (2)","Quarterly (4)","Bimonthly (6)","Monthly (12)","Biweekly (26)","Weekly (52)","Semiweekly (104)",],rotation=30,)
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

#plot_periodogram(aggsale_train)

plot_periodogram(truncated_whole_sales)

This is more similar to an expected periodogram. There seems to be a weekly component to sales, as well as monthly, bimonthly, quarterly, semiannually and annual patterns.
The next step is to create a model to capture these features, take a look at the model fit, and then use the model's prediction to confirm the periodogram is deseasoned when these features are implemented.
The first step here will be to split the model into a training set and a test set, such that time series predictions can be verified. For these purposes, 15 days of predicition should be sufficient.


In [ ]:
whole.dtypes

In [ ]:
# Count the number of 0.0 values in a specific column
number_of_zeros = (whole['Quantity'] == 0.0).sum()
print(f"Number of 0.0 values in column '{'Quantity'}': {number_of_zeros}")


In [ ]:
whole.set_index('Date', inplace=True)
#sort by the index
whole.sort_index(inplace=True)
#slice to set up train and test

train= whole.loc['2018-04-26':'2020-03-02']
test= whole.loc['2020-03-03':'2020-03-17']

aggsale_train = train['Sales'].groupby(train.index).sum()
aggsale_test = test['Sales'].groupby(test.index).sum()

aggsale_train.head()


First convert the dates to periods with a frequency of daily for use in the model

In [ ]:
aggsale_train = aggsale_train.asfreq('D')
print(aggsale_train.index.freq)

In [ ]:
print(aggsale_train.isna().sum())

There are 19 dates added to the time series that were previously missing. Take look to see if there is a pattern to missing dates.

In [ ]:
aggsale_train = aggsale_train.reset_index()
aggsale_train.columns = ['Date', 'Sales']

nan_dates = aggsale_train[aggsale_train['Sales'].isna()]['Date']
print(nan_dates)

Some of these dates align with holiday store closures, Such as 12-25, 07-04. Others may relate to store closures near Thanksgiving and Labor Day. The best way to deal with these nan (no entry in the sales column) is to convert them to 0's, indicating no sales.

In [ ]:
aggsale_train['Sales'].fillna(0, inplace = True)
aggsale_train.isna().sum()


In [ ]:
#Convert dataframe back to periodic frequency
aggsale_train.set_index('Date', inplace=True)
aggsale_train = aggsale_train.asfreq('D')
print(aggsale_train.index.freq)

In [ ]:
#create a model
y = aggsale_train.copy()
fourier = CalendarFourier(freq='M', order=4)
fourier_annual = CalendarFourier(freq='A', order=4)
fourier_weekly = CalendarFourier(freq='W', order=4)
fourier_quarterly = CalendarFourier(freq='Q', order=4)
# Bimonthly Fourier
fourier_bimonthly = CalendarFourier(freq='2M', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier,fourier_weekly,fourier_annual,fourier_quarterly, fourier_bimonthly],
    drop=True,
)
X = dp.in_sample()

In [ ]:
#Run the model
model = LinearRegression().fit(X, y)
y_pred = pd.Series(
    model.predict(X)[:, 0], #this flattens by slicing
    index=X.index,
    name='Fitted',
)

ax = y.plot(**plot_params, alpha=0.5, title="Average Sales", ylabel="items sold")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax.legend();

Root Mean Squared Logrithmin Error (RSMLE) is a method of measuring the error in the curve generated in the model compared to original samples. 


In [ ]:
def rmsle(y_true, y_pred):
    
    # Ensure the arrays are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Compute the squared logarithmic error
    error = (np.log(y_pred + 1) - np.log(y_true + 1)) ** 2
    
    # Return the square root of the mean of the squared logarithmic error
    return np.sqrt(np.mean(error))

In [ ]:
in_sample_rmsle = rmsle(y, y_pred)
print(f"In-sample RMSLE: {in_sample_rmsle}")

RSMLE values are relative, it will be compared to out-sample (future prediction generated from the test set) as well as further modifications for the model. A lower value is lower error in the model.
Next, verify that the main features of the periodogram have been captured by the model, by generating a priodogram with the model substrated.

In [ ]:
y_squ = np.squeeze(y)
print(y_squ.shape)
print(X.shape)

In [ ]:
#deseason the data and look at the periodogram to see if any more seasonailty not captured
y_deseason = y_squ - y_pred

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))
ax1 = plot_periodogram(y_squ, ax=ax1)
ax1.set_title("Product Sales Frequency Components")
ax2 = plot_periodogram(y_deseason, ax=ax2);
ax2.set_title("Deseasonalized");
#it appears that seaonality of the slaes is captured

The seasonality of the data looks to be covered in the model. 
Now, exploring additional features to add to the model.
Starting by taking a look at days of the week, days of the month, sales across weeks, months and years to look for any trends that may not have been captured in the model.

In [ ]:
aggsale_train = aggsale_train.reset_index()
print(aggsale_train.columns)
aggsale_train.head()

In [ ]:

COLORS = list(sns.color_palette())
def plot_date_distribution():   
    
    df = aggsale_train.copy().sort_values(["Date"], ignore_index=True)
    df["day"] = df.Date.dt.day
    df["month"] = df.Date.dt.month
    df["year"] = df.Date.dt.year
    df["day_of_week"] = df.Date.dt.dayofweek
    df["day_of_year"] = df.Date.dt.dayofyear
    df["week_of_year"] = df.Date.dt.isocalendar().week.astype(int)
    df["date_index"] = df.Date.factorize()[0]
    plot_kwargs = {
        "linewidth": 2,
        "flierprops": {"alpha": 0.2},
        "orient": "h",}
    
    fig = plt.figure(figsize=(18, 24))
    gs = fig.add_gridspec(5, 2, height_ratios=(3, 5, 2, 5, 5))
    sns.boxplot(
        data=df,
        y="day_of_week",
        x="Sales",
        color=COLORS[0],
        ax=fig.add_subplot(gs[0, 0]),
        **plot_kwargs,
    )
    plt.title("Distribution of Sales across the Days of the Week")
    
    sns.boxplot(
        data=df,
        y="month",
        x="Sales",
        color=COLORS[1],
        ax=fig.add_subplot(gs[1, 0]),
        **plot_kwargs,
         )
    plt.title("Distribution of Sales across the Months")
    
    sns.boxplot(
        data=df,
        y="year",
        x="Sales",
        color=COLORS[2],
        ax=fig.add_subplot(gs[2, 0]),
        **plot_kwargs,
    )
    plt.title("Distribution of Sales across the Years")
    
    sns.boxplot(
        data=df,
        y="day",
        x="Sales",
        color=COLORS[3],
                ax=fig.add_subplot(gs[:3, 1]),
        **plot_kwargs,
    )
    plt.title("Distribution of Sales across the Days of the Month")
    sns.lineplot(
        data=df.groupby("day_of_year").Sales.mean().reset_index(),
        x="day_of_year",
        y="Sales",
        color=COLORS[4],
        ax=fig.add_subplot(gs[3, 0]),
        linewidth=2,
    )
    plt.ylabel("Average Sales")
    plt.title("Average Sales across the Days of the Year")
    
    sns.lineplot(
        data=df.groupby("week_of_year").Sales.mean().reset_index(),
        x="week_of_year",
        y="Sales",
        color=COLORS[5],
        ax=fig.add_subplot(gs[3, 1]),
        linewidth=2,
    )
    plt.ylabel("Average Sales")
    plt.title("Average Sales across the Weeks of the Year")
        
    sns.lineplot(
        data=df.groupby("date_index").Sales.mean().reset_index(),
        x="date_index",
        y="Sales",
        color=COLORS[6],
        ax=fig.add_subplot(gs[4, :]),
        linewidth=2,
    )
    plt.ylabel("Average Sales")
    plt.title("Average Sales across the Date Index")
    
    plt.tight_layout()
    plt.show()
    
plot_date_distribution()
    

The sales are higher on Saturdays, and then Sundays and Mondays. The day of the week feature should already covered by the weekly feature in the model (this was confirmed separately and addition of a day of the week feature did not affect the model, so was omitted). There is no particular trend apparent in the day of the month or yearly graphs. Sales are highest Nov-Dec, as expected, and this is likely covered with the annual feature in the model.

Moving forward, adding in specific holidays that repeat annually, adding in days with 0 sales (store closed) as a feature, and then investigating features exploring sales categories, brands and products.

In [ ]:
y_deseason = y_squ - y_pred
average_sales_value = y_deseason.median() 
holiday_deviations = abs(y_deseason[aggsale_train.index] - average_sales_value)
impactful_holidays = holiday_deviations.sort_values(ascending=False)
impactful_holidays.head() 


In [ ]:
impactful_holidays = impactful_holidays.reset_index()
print(impactful_holidays.columns)
impactful_holidays.head(20)



In [ ]:

#Create a 'month_day' column to hold the month and day
impactful_holidays['month_day'] = impactful_holidays['Date'].dt.strftime('%m-%d')

# Sort the dataframe by 'month_day' and the impact scores in descending order
impactful_holidays_sorted = impactful_holidays.sort_values(by=['month_day', 0], ascending=[True, False])


# This function will check if there are high impact scores on the same 'month_day' across different years
def filter_high_impact_days(group):
    if group['Date'].dt.year.nunique() > 1:  # Checking if there are entries from multiple years
        return group

# Apply the filter function to each group
high_impact_dates = impactful_holidays_sorted.groupby('month_day').apply(filter_high_impact_days)

# If you want to reset the index to get a clean dataframe
high_impact_dates.reset_index(drop=True, inplace=True)

print(high_impact_dates)


In [ ]:
# Group by 'month_day' and get the maximum impact score for each group
max_impact_scores = high_impact_dates.groupby('month_day')[0].max().reset_index()

# Sort the result by the impact score in descending order to have the highest impacts at the top
max_impact_scores_sorted = max_impact_scores.sort_values(by=0, ascending=False).reset_index(drop=True)

# Now max_impact_scores_sorted will have the highest impact score for each month-day, sorted from highest to lowest
max_impact_scores_sorted.head(30)


In [ ]:
#it looks like there are high impact sales in the days leading up to Christmas, determine which days to use
start_date = '12-14'
end_date = '12-25'

# Filter the DataFrame based on the date range
filtered_df = max_impact_scores_sorted[(max_impact_scores_sorted['month_day'] >= start_date) & (max_impact_scores_sorted['month_day'] <= end_date)]

print(filtered_df)
# sorting through excel reveals that the average impact is highest 12-21 thru 12-25 both yea

In [ ]:
aggsale_train['Date'].dtypes

In [ ]:
# take a look to confirm that sales where far above or below median on particular dates as indicated above

particular_date = '2019-12-18'

# Convert the string to a datetime object
particular_date = pd.to_datetime(particular_date)

# Use loc to filter the dataframe for the particular date
sales_on_particular_date = aggsale_train.loc[aggsale_train['Date'] == particular_date]

print(sales_on_particular_date)

It looks like December 20-25 could have an impact on the model. Additionally, December 18 and 14 could have an impact. 
The model will be run with and without these additional dates to see the impact on the RSMLE, but first, more features.  

In [ ]:
#new column called store closed that has a 1 when sales = 0  
aggsale_train['Store_closed'] = (aggsale_train['Sales'] == 0).astype(int)
aggsale_train.head()


In [ ]:
# now, do the same for aggsale_test, convert to dataframe first
aggsale_test = aggsale_test.reset_index()
#print(aggsale_test.columns)
aggsale_test['Store_closed'] = (aggsale_test['Sales'] == 0).astype(int)
aggsale_test.head()

Next, exploring features based on Account categories.
create account_type column in train df ( the df filtered for dates from whole, contains all columns), get dummies, then merge with aggsale_train grouped by date with sum or sales and of each account_type column

In [ ]:

train_at = train
train_at = train_at.drop(['Item', 'COGS Amount','Est. Unit Cost','Category','Product Line'], axis=1)
train_at.head()



In [ ]:
conditions = [
    train_at['Account'].str.contains('Inventory', case=False, na=False),
    train_at['Account'].str.contains('Cost of Goods', case=False, na=False),
    train_at['Account'].str.contains('Logistics', case=False, na=False),    
    train_at['Account'].str.contains('Revenue', case=False, na=False) & train['Account'].str.contains('candles', case=False, na=False),
    train_at['Account'].str.contains('Revenue', case=False, na=False) & train['Account'].str.contains('Accessories', case=False, na=False),
    train_at['Account'].str.contains('Workshop', case=False, na=False),
    ]

# Define the choices corresponding to conditions
choices = [
    'Inventory',
    'Cost of Goods',
    'Logistics',
    'Revenue candles',
    'Revenue Accessories',
    'Revenue Workshop',
    ]

# Create the new column using np.select()

train_at['Account_type'] = np.select(conditions, choices, default=np.nan)


train_at.head()

In [ ]:
test_at = test
conditions = [
    test_at['Account'].str.contains('Inventory', case=False, na=False),
    test_at['Account'].str.contains('Cost of Goods', case=False, na=False),
    test_at['Account'].str.contains('Logistics', case=False, na=False),    
    test_at['Account'].str.contains('Revenue', case=False, na=False) & test_at['Account'].str.contains('candles', case=False, na=False),
    test_at['Account'].str.contains('Revenue', case=False, na=False) & test_at['Account'].str.contains('Accessories', case=False, na=False),
    test_at['Account'].str.contains('Workshop', case=False, na=False),
    ]

# Define the choices corresponding to conditions
choices = [
    'Inventory',
    'Cost of Goods',
    'Logistics',
    'Revenue candles',
    'Revenue Accessories',
    'Revenue Workshop',
    
]

# Create the new column using np.select()

test_at['Account_type'] = np.select(conditions, choices, default=np.nan)


test_at.head()

In [ ]:
train_at['Account_type'].unique().tolist()

In [ ]:
# Replace NaN with a placeholder value,'Unknown', then create dummies and concat
train_at['Account_type'] = train_at['Account_type'].fillna('Unknown')
test_at['Account_type'] = test_at['Account_type'].fillna('Unknown')
account_type_dummies = pd.get_dummies(train_at['Account_type']).astype(int)
account_type_dummies_test = pd.get_dummies(test_at['Account_type']).astype(int)


account_type_dummies.head()

In [ ]:

train_gr = train_at.drop(['Quantity','Account','Description'], axis =1)
train_gr.head()

In [ ]:

number_of_revenue_workshops = (train_gr['Account_type'] == 'Revenue Workshop').sum()
print(f"Number of 'Revenue Workshop' entries: {number_of_revenue_workshops}")


In [ ]:

def plot_date_distribution(df):
    
    df_red = df[(df['Account_type'] != 'Cost of Goods')  & (df['Account_type'] != 'nan')]

    plt.figure(figsize=(9, 5))
    ax = sns.boxplot(
        data=df_red,
        x="Sales",
        y="Account_type",
        palette="viridis"
    )
    
    q1 = df_red['Sales'].quantile(0.25)
    q3 = df_red['Sales'].quantile(0.75)
    iqr = q3 - q1

    
    #ax.set_xlim([q1 - 1.5 * iqr, q3 + 1.5 * iqr])
    ax.set_xlim([0, q3 + 1.5 * iqr])

    plt.title("Distribution of Sales by Account Type")
    plt.xlabel("Sales")
    plt.ylabel("Account Type")
    plt.show()

# Call the function with your data
plot_date_distribution(train_gr)


    

In [ ]:
# Inspect the data for "Workshops"
workshops_data = train_gr[train_gr['Account_type'] == 'Revenue Workshop']
print(workshops_data['Sales'].describe())


In [ ]:
# Filter to find the specific rows where 'Account_type' is 'Revenue Workshop' and 'Sales' is 2272
outliers = train_gr[(train_gr['Account_type'] == 'Revenue Workshop') & (train_gr['Sales'] >= 800)]

# Print the outlier rows to investigate
print(outliers)


Revenue workshops seemed to have high values, but they appear to be legitimate. Try adjusting to log scale to adjust for these high values.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
ax = sns.boxplot(x='Account_type', y='Sales', data=train_gr, palette='Set2')

# Set the y-axis to a logarithmic scale
ax.set_yscale('log')

plt.xticks([0,1,2,3,4,5,6], ["Candles", "Cost of Goods", "Inventory",'Accessories','Other','Workshops','Logistics'])
plt.title('Sales Distribution by Account Type')
plt.xlabel('Account Type')
plt.ylabel('Sales')
plt.show()


In [ ]:

df_filtered = train_gr[(train_gr['Account_type'] != 'Cost of Goods') & 
                       (train_gr['Account_type'] != 'Inventory') &
                       (train_gr['Account_type'] != 'nan') &
                       (train_gr['Account_type'] != 'Logistics') &
                       (~pd.isna(train_gr['Account_type']))]

# Now create the plot using df_filtered
plt.figure(figsize=(8, 5))
ax = sns.boxplot(x='Sales', y='Account_type', data=df_filtered, palette='Set2')

# Set the y-axis to a logarithmic scale
ax.set_xscale('log')

plt.yticks([0,1,2], ["Candles", 'Accessories','Workshops'])
plt.title('Sales Distribution by Account Type')
plt.xlabel('Sales')
plt.ylabel('Account Type')
plt.show()


In [ ]:
# Aggregate the dummies at a daily level
account_dummies_agg = account_type_dummies.groupby('Date').sum() 
account_dummies_agg_t = account_type_dummies_test.groupby('Date').sum() 

account_dummies_agg.head()

In [ ]:
account_dummies_agg = account_dummies_agg.reset_index()
account_dummies_agg_t = account_dummies_agg_t.reset_index()
account_dummies_agg_t.head()

In [ ]:
# Merge
aggsale_train= aggsale_train.merge(account_dummies_agg, on='Date', how='left')
aggsale_test= aggsale_test.merge(account_dummies_agg_t, on='Date', how='left')
aggsale_train.head()


Next, adding features for Brands and Product type. There are 135 unique brands in the dataset. In order to see the top ten brands by sales, group the date by brand and sum the sales for each category. 

In [ ]:
aggsale_test.head()

In [ ]:
train['Brand'].nunique()

In [ ]:
brand_sales_sum = train_gr.groupby('Brand')['Sales'].sum()

# Sort the sums in descending order and get the top 10
top_ten_brands = brand_sales_sum.sort_values(ascending=False).head(10)

# Print the top 10 brands
print(top_ten_brands)


In [ ]:
brand_sales_sumt = test.groupby('Brand')['Sales'].sum()
top_ten_brandst = brand_sales_sumt.sort_values(ascending=False).head(10)


print(top_ten_brandst)


In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(x=top_ten_brands.values, y=top_ten_brands.index)
plt.title('Top Ten Selling Brands')
plt.xlabel('Total Sales')
plt.ylabel('Brand')
plt.show()


Now, make a feature with the brands that are in common for both the training and test datasets

In [ ]:
set1 = set(top_ten_brands.index)
set2 = set(top_ten_brandst.index)

common_brands = set1.intersection(set2)
common_brands_list = list(common_brands)
print(common_brands_list)


In [ ]:
def categorize_brand(brand):
    if brand in common_brands_list:
        return brand
    else:
        return 'Other'

# Apply the function to the 'Brand' column
train_at['Common_Brand'] = train_at['Brand'].apply(categorize_brand)
test_at['Common_Brand'] = test_at['Brand'].apply(categorize_brand)

# Create dummy variables
brand_dummies_train = pd.get_dummies(train_at['Common_Brand']).astype(int)
brand_dummies_test = pd.get_dummies(test_at['Common_Brand']).astype(int)

# Checking the result
brand_dummies_train.head()


In [ ]:
brand_dummies_train = brand_dummies_train.drop(['Other'], axis =1)
brand_dummies_test = brand_dummies_test.drop(['Other'], axis =1)
brand_dummies_train.head()

In [ ]:
brand_dummies_test.head()

In [ ]:
#aggregate by sales per day, reset inddex and then merge
brand_dummies_agg = brand_dummies_train.groupby('Date').sum() 
brand_dummies_agg_t = brand_dummies_test.groupby('Date').sum() 
brand_dummies_agg = brand_dummies_agg.reset_index()
brand_dummies_agg_t = brand_dummies_agg_t.reset_index()

brand_dummies_agg.head()

In [ ]:
brand_dummies_agg_t.head()

In [ ]:
aggsale_train= aggsale_train.merge(brand_dummies_agg, on='Date', how='left')
aggsale_test= aggsale_test.merge(brand_dummies_agg_t, on='Date', how='left')
aggsale_train.head()


In [ ]:
aggsale_test.head()

In [ ]:
train['Description'].nunique()

In [ ]:
train.head()

In [ ]:

desc_sales_sum = train.groupby('Description')['Sales'].sum()
top_desc = desc_sales_sum.sort_values(ascending=False).head(100)

print(top_desc)





In [ ]:
import re
#Convert the series to a DataFrame
top_desc_df = top_desc.reset_index()
top_desc_df.columns = ['Description', 'Sales']


def combine_specific_descriptions(desc):
    if re.search(r'No\. 4(\D|$)', desc):
        return 'No. 4'
    elif re.search(r'No\. 8(\D|$)', desc):
        return 'No. 8'
    elif re.search(r'No\. 25(\D|$)', desc):
        return 'No. 25'
    elif re.search(r'No\. 12(\D|$)', desc):
        return 'No. 12'
    elif re.search(r'No\. 70(\D|$)', desc):
        return 'No. 70'
    elif re.search(r'No\. 9(\D|$)', desc):
        return 'No. 9'
    elif re.search(r'No\. 52(\D|$)', desc):
        return 'No. 52'
    elif re.search(r'No\. 71(\D|$)', desc):
        return 'No. 71'
    elif re.search(r'No\. 88(\D|$)', desc):
        return 'No. 88'
    elif re.search(r'No\. 40(\D|$)', desc):
        return 'No. 40'
    elif re.search(r'No\. 18(\D|$)', desc):
        return 'No. 18'
    elif re.search(r'No\. 1(\D|$)', desc):
        return 'No. 1'
    elif re.search(r'No\. 31(\D|$)', desc):
        return 'No. 31'
    elif re.search(r'No\. 100(\D|$)', desc):
        return 'No. 100'
    elif re.search(r'No\. 67(\D|$)', desc):
        return 'No. 67'
    elif re.search(r'No\. 59(\D|$)', desc):
        return 'No. 59'
    elif re.search(r'No\. 72(\D|$)', desc):
        return 'No. 72'
    elif re.search(r'No\. 68(\D|$)', desc):
        return 'No. 68'
    elif re.search(r'No\. 64(\D|$)', desc):
        return 'No. 64'
    elif re.search(r'No\. 83(\D|$)', desc):
        return 'No. 83'
    elif re.search(r'No\. 53(\D|$)', desc):
        return 'No. 53'
    elif re.search(r'No\. 76(\D|$)', desc):
        return 'No. 76'
    elif re.search(r'No\. 90(\D|$)', desc):
        return 'No. 90'
    else:
        return desc

# Apply this function to the 'Description' column
top_desc_df['Description'] = top_desc_df['Description'].apply(combine_specific_descriptions)

# Group by the new description and sum the sales
combined_desc_sales = top_desc_df.groupby('Description')['Sales'].sum()

# Sort the values if needed
combined_desc_sales = combined_desc_sales.sort_values(ascending=False)

combined_desc_sales.head(50)



In [ ]:
combined_desc_saleslim = combined_desc_sales.head(10)
plt.figure(figsize=(8, 5))
sns.barplot(x=combined_desc_saleslim.values, y=combined_desc_saleslim.index)

plt.title('Top Ten Selling Items')
plt.xlabel('Total Sales')
plt.ylabel('Item')
plt.show()

The difference between items is very small after the first few. Setting these up as features to see if they have an impact on the model. 

In [ ]:

desc_test = test.groupby('Description')['Sales'].sum()
desc_test = desc_test.sort_values(ascending=False).head(10)

print(desc_test)

In [ ]:
# combine the similar items
desc_test_df = desc_test.reset_index()
desc_test_df.columns = ['Description', 'Sales']


def combine_specific_descriptions(desc):
    if re.search(r'No\. 4(\D|$)', desc):
        return 'No. 4'
    elif re.search(r'No\. 8(\D|$)', desc):
        return 'No. 8'
    elif re.search(r'No\. 25(\D|$)', desc):
        return 'No. 25'
    elif re.search(r'No\. 12(\D|$)', desc):
        return 'No. 12'
    elif re.search(r'No\. 70(\D|$)', desc):
        return 'No. 70'
    elif re.search(r'No\. 9(\D|$)', desc):
        return 'No. 9'
    elif re.search(r'No\. 52(\D|$)', desc):
        return 'No. 52'
    elif re.search(r'No\. 71(\D|$)', desc):
        return 'No. 71'
    elif re.search(r'No\. 88(\D|$)', desc):
        return 'No. 88'
    elif re.search(r'No\. 40(\D|$)', desc):
        return 'No. 40'
    elif re.search(r'No\. 18(\D|$)', desc):
        return 'No. 18'
    elif re.search(r'No\. 1(\D|$)', desc):
        return 'No. 1'
    elif re.search(r'No\. 31(\D|$)', desc):
        return 'No. 31'
    elif re.search(r'No\. 100(\D|$)', desc):
        return 'No. 100'
    elif re.search(r'No\. 67(\D|$)', desc):
        return 'No. 67'
    elif re.search(r'No\. 59(\D|$)', desc):
        return 'No. 59'
    elif re.search(r'No\. 72(\D|$)', desc):
        return 'No. 72'
    elif re.search(r'No\. 68(\D|$)', desc):
        return 'No. 68'
    elif re.search(r'No\. 64(\D|$)', desc):
        return 'No. 64'
    elif re.search(r'No\. 83(\D|$)', desc):
        return 'No. 83'
    elif re.search(r'No\. 53(\D|$)', desc):
        return 'No. 53'
    elif re.search(r'No\. 76(\D|$)', desc):
        return 'No. 76'
    elif re.search(r'No\. 90(\D|$)', desc):
        return 'No. 90'
    else:
        return desc


desc_test_df['Description'] = desc_test_df['Description'].apply(combine_specific_descriptions)

# Group by the new description and sum the sales
combined_desc_sales = desc_test_df.groupby('Description')['Sales'].sum()

# Sort the values if needed
combined_desc_salestest = combined_desc_sales.sort_values(ascending=False)

combined_desc_salestest.head(10)

In [ ]:
combined_desc_salestest=combined_desc_salestest.head(10)

set1 = set(combined_desc_saleslim.index)
set2 = set(combined_desc_salestest.index)

common_items = set1.intersection(set2)
common_items_list = list(common_items)
print(common_items_list)

These are the brands in common in the top ten for the training and test sets. 


In [ ]:

common_items_list.append('Shipping')
def categorize_item(item):
    item_str = str(item)
    for common_item in common_items_list:
        if common_item in item_str:
            return common_item
    return 'Other'

train_at['Common_Item'] = train_at['Description'].apply(categorize_item)
test_at['Common_Item'] = test_at['Description'].apply(categorize_item)


# Create dummy variables
item_dummies_train = pd.get_dummies(train_at['Common_Item']).astype(int)
item_dummies_test = pd.get_dummies(test_at['Common_Item']).astype(int)

# Checking the result
item_dummies_train.head()


In [ ]:
item_dummies_test.head()

In [ ]:
#shipping
train_at['Common_Item'] = train_at['Description'].apply(categorize_item)
test_at['Common_Item'] = test_at['Description'].apply(categorize_item)


# Create dummy variables
item_dummies_train = pd.get_dummies(train_at['Common_Item']).astype(int)
item_dummies_test = pd.get_dummies(test_at['Common_Item']).astype(int)

In [ ]:
item_dummies_train = item_dummies_train.drop(['Other'], axis =1)
item_dummies_test = item_dummies_test.drop(['Other'], axis =1)
item_dummies_train.head()

In [ ]:
item_dummies_test.head()

In [ ]:
item_dummies_train_agg = item_dummies_train.groupby('Date').sum()
item_dummies_test_agg = item_dummies_test.groupby('Date').sum()
item_dummies_train_agg = item_dummies_train_agg.reset_index()
item_dummies_test_agg = item_dummies_test_agg.reset_index()
item_dummies_train_agg.head()

In [ ]:
item_dummies_test_agg.head()

In [ ]:
aggsale_test.head()

In [ ]:
aggsale_train = aggsale_train.merge(item_dummies_train_agg, on='Date', how='left')
aggsale_test = aggsale_test.merge(item_dummies_test_agg, on='Date', how='left')
aggsale_train.head()

In [ ]:
aggsale_test.head()

In [ ]:
aggsale_train.set_index('Date', inplace=True)
aggregated_train = aggsale_train.groupby('Date').sum()
aggsale_test.set_index('Date', inplace=True)
aggregated_test = aggsale_test.groupby('Date').sum()

# Check the result
aggregated_test.head()


In [ ]:
aggsale_train = aggsale_train.asfreq('D')
aggsale_test = aggsale_test.asfreq('D')

# Fill NaN values with 0
aggsale_train.fillna(0, inplace=True)
aggsale_test.fillna(0, inplace=True)
aggsale_train.isna().sum()

In [ ]:
aggsale_test.isna().sum()

In [ ]:
y = aggsale_train.copy()

# Applying log transformation to the 'Sales' column
y_log = np.log1p(y['Sales'])

fourier = CalendarFourier(freq='M', order=4)
fourier_annual = CalendarFourier(freq='A', order=4)#add weekly too, only has monthly, weird
fourier_weekly = CalendarFourier(freq='W', order=4)
fourier_quarterly = CalendarFourier(freq='Q', order=4)# Quarterly Fourier
# Bimonthly Fourier
fourier_bimonthly = CalendarFourier(freq='2M', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier,fourier_weekly,fourier_annual,fourier_quarterly, fourier_bimonthly],
    drop=True,
)
X = dp.in_sample()


X['Xmas25'] = ((X.index.month == 12) & (X.index.day == 25)).astype(int)
X['Xmas24'] = ((X.index.month == 12) & (X.index.day == 24)).astype(int)
X['Xmas23'] = ((X.index.month == 12) & (X.index.day == 23)).astype(int)
X['Xmas22'] = ((X.index.month == 12) & (X.index.day == 22)).astype(int)
X['Xmas21'] = ((X.index.month == 12) & (X.index.day == 21)).astype(int)
X['Xmas20'] = ((X.index.month == 12) & (X.index.day == 20)).astype(int)
X['Xmas18'] = ((X.index.month == 12) & (X.index.day == 18)).astype(int)
X['Xmas14'] = ((X.index.month == 12) & (X.index.day == 14)).astype(int)
X['Store_closed'] = aggsale_train['Store_closed']

X['Cost of Goods'] = aggsale_train['Cost of Goods']
#X['Inventory'] = aggsale_train['Inventory']
X['Logistics'] = aggsale_train['Logistics']
X['Revenue Accessories'] = aggsale_train['Revenue Accessories']
X['Revenue candles'] = aggsale_train['Revenue candles']
X['Revenue Workshop'] = aggsale_train['Revenue Workshop']
X['nan'] = aggsale_train['nan']

X['Cozy Corner'] = aggsale_train['Cozy Corner']
X['Petalume'] = aggsale_train['Petalume']
X['Bliss Bungalow'] = aggsale_train['Bliss Bungalow']
X['AromaGarden'] = aggsale_train['AromaGarden']
X['Hearth Harbor'] = aggsale_train['Hearth Harbor']
X['Meadow Mingle'] = aggsale_train['Meadow Mingle']

#X['No. 8'] = aggsale_train['No. 8']
#X['AromaGarden Gardenia candle'] = aggsale_train['AromaGarden Gardenia candle']
#X['No. 31'] = aggsale_train['No. 31']
#X['No. 25'] = aggsale_train['No. 25']
X['Shipping'] = aggsale_train['Shipping']

model = LinearRegression(fit_intercept=False)
#model.fit(X, y)
model.fit(X, y_log)

y_pred_log = model.predict(X)
y_pred_insample = np.expm1(y_pred_log)  # Inverse of log1p

# Creating a DataFrame for the predicted values
y_pred_insample= pd.DataFrame(y_pred_insample, index=X.index, columns=['Sales'])


#model.fit(X, y['Sales'])
#y_pred_insample = pd.DataFrame(model.predict(X), index=X.index, columns=['Sales'])
#y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [ ]:

def rmsle(y_true, y_pred):
    
    # Ensure the arrays are numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Compute the squared logarithmic error
    error = (np.log(y_pred + 1) - np.log(y_true + 1)) ** 2
    
    # Return the square root of the mean of the squared logarithmic error
    return np.sqrt(np.mean(error))



In [ ]:
# there are negative predictions in the data, for now clip them to 0 or low pos near 0 to make the calc work
# Clip negative predictions to 0

# Now, compute RMSLE
in_sample_rmsle = rmsle(y['Sales'], y_pred_insample['Sales'].clip(lower=0))
print(f"In-sample RMSLE: {in_sample_rmsle}")





In [ ]:
#creates feature set for the forecast data (out_of_sample) 

X_test = dp.out_of_sample(steps=15)
X_test['Xmas25'] = ((X_test.index.month == 12) & (X_test.index.day == 25)).astype(int)
X_test['Xmas24'] = ((X_test.index.month == 12) & (X_test.index.day == 24)).astype(int)
X_test['Xmas23'] = ((X_test.index.month == 12) & (X_test.index.day == 23)).astype(int)
X_test['Xmas22'] = ((X_test.index.month == 12) & (X_test.index.day == 22)).astype(int)
X_test['Xmas21'] = ((X_test.index.month == 12) & (X_test.index.day == 21)).astype(int)
X_test['Xmas20'] = ((X_test.index.month == 12) & (X_test.index.day == 20)).astype(int)
X_test['Xmas18'] = ((X_test.index.month == 12) & (X_test.index.day == 18)).astype(int)
X_test['Xmas14'] = ((X_test.index.month == 12) & (X_test.index.day == 14)).astype(int)
X_test['Store_closed'] = aggsale_test['Store_closed']

X_test['Cost of Goods'] = aggsale_test['Cost of Goods']
#X_test['Inventory'] = aggsale_test['Inventory']
X_test['Logistics'] = aggsale_test['Logistics']
X_test['Revenue Accessories'] = aggsale_test['Revenue Accessories']
X_test['Revenue candles'] = aggsale_test['Revenue candles']
X_test['Revenue Workshop'] = aggsale_test['Revenue Workshop']
X_test['nan'] = aggsale_test['nan']

X_test['Cozy Corner'] = aggsale_test['Cozy Corner']
X_test['Petalume'] = aggsale_test['Petalume']
X_test['Bliss Bungalow'] = aggsale_test['Bliss Bungalow']
X_test['AromaGarden'] = aggsale_test['AromaGarden']
X_test['Hearth Harbor'] = aggsale_test['Hearth Harbor']
X_test['Meadow Mingle'] = aggsale_test['Meadow Mingle']

#X_test['No. 8'] = aggsale_test['No. 8']
#X_test['AromaGarden Gardenia candle'] = aggsale_test['AromaGarden Gardenia candle']
#X_test['No. 31'] = aggsale_test['No. 31']
#X_test['No. 25'] = aggsale_test['No. 25']
X_test['Shipping'] = aggsale_test['Shipping']




In [ ]:
# Predict using the model on the out-of-sample data
y_pred_outsample_log = model.predict(X_test)


y_pred_outsample = np.expm1(y_pred_outsample_log).flatten()  # Flatten if it's a 2D array with one column
y_pred_outsample_df = pd.DataFrame(y_pred_outsample, index=X_test.index, columns=['Sales'])

out_sample_rmsle = rmsle(aggsale_test['Sales'], y_pred_outsample_df['Sales'].clip(lower=0))
print(f"Out-of-sample RMSLE: {out_sample_rmsle}")
#model = LinearRegression(fit_intercept=False)
#model.fit(X, y_log)  .



# Applying inverse log transformation to the predictions
#y_pred_outsample = np.expm1(y_pred_outsample_log)

# Creating a DataFrame for the predicted values
#y_pred_outsample_df = pd.DataFrame(y_pred_outsample, index=X_test.index, columns=['Sales'])

# Calculate RMSLE for Out-of-Sample Data
# Make sure aggsale_test['Sales'] is the actual sales data corresponding to the out-of-sample period
#out_sample_rmsle = rmsle(aggsale_test['Sales'], y_pred_outsample_df['Sales'].clip(lower=0))
#print(f"Out-of-sample RMSLE: {out_sample_rmsle}")


In [ ]:
##model = LinearRegression(fit_intercept=False)
##model.fit(X, y['Sales'])
#y_pred_outsample = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=['Sales'])



# Creating a DataFrame for the predicted values
#y_pred_outsample = pd.DataFrame(y_pred_outsample, index=X_test.index, columns=['Sales'])



In [ ]:
#out_sample_rmsle = rmsle(aggsale_test['Sales'], y_pred_outsample['Sales'])
#print(f"Out-of-sample RMSLE: {out_sample_rmsle}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y['Sales'], label='Training Data', color='blue')
plt.plot(y_pred_insample['Sales'], label='Model', color='orange')
plt.title('In-Sample Sales Prediction')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))

# Plotting the predicted out-of-sample sales
plt.plot(y_pred_outsample_df.index, y_pred_outsample_df['Sales'], label='Prediction', color='green')

# Plotting the actual out-of-sample sales
plt.plot(aggsale_test.index, aggsale_test['Sales'], label='Test data', color='red')

plt.title('Sales Prediction')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y['Sales'], label='Training data', color='blue')
#plt.plot(aggsale_test['Sales'], label='Test data', color='red')

plt.plot(y_pred_insample['Sales'], label='Model', color='orange')
#plt.plot(y_pred_outsample['Sales'], label='Prediction', color='green')
plt.plot(y_pred_outsample_df.index, y_pred_outsample_df['Sales'], label='Prediction', color='green')

# Plotting the actual out-of-sample sales
plt.plot(aggsale_test.index, aggsale_test['Sales'], label='Test data', color='red')



plt.title('Sales Model and Prediction')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()


In [ ]:
data = {
    'Model Features': ['Dec 20-25', 
                       'Dec 14,18,20-25',
                       'Dec 14,18,20-25, Store Closed',
                       'Dec 14,18,20-25, Store Closed,Tday, 4th',
                       'Dec 14,18,20-25, Store Closed, COGS',
                       'Dec 14,18,20-25, Store Closed, All Account Type',
                       'Dec 14,18,20-25, Store Closed, Account Type, Brands',
                       'Dec 14,18,20-25, Store Closed, Account Type, Product Types',
                       'Log Transformation Dec14, 18,20-25, Store Closed, Account Type'],
    'RSMLE training data': ['1.44', '1.43','0.90','0.93','0.58','0.58','0.58','0.58','0.30'],
    'RSMLE prediction': ['0.59', '0.57','0.55','0.55','0.18','0.13','0.13','0.13','0.24']
}
RSMLE_model = pd.DataFrame(data)


RSMLE_model

In [ ]:
aggsale_train.head()


In [ ]:
#Filter training data
X_train_cozy_corner = X[X['Cozy Corner'] > 0]
y_train_cozy_corner = y[X['Cozy Corner'] >0 ]

# Filter the test data
X_test_cozy_corner = X_test[X_test['Cozy Corner'] >0]
y_test_cozy_corner = aggsale_test[X_test['Cozy Corner'] >0]['Sales']  # Assuming 'Sales' is your target variable

# Ensure y_test_cozy_corner is a 1D array
y_test_cozy_corner = y_test_cozy_corner.values.ravel()  # Converts to numpy array and flattens it

model = LinearRegression(fit_intercept=False)
model.fit(X_train_cozy_corner, y_train_cozy_corner)

# Predict and select the correct column 
y_pred_cozy_corner = model.predict(X_test_cozy_corner)[:, 0]  
# Calculate RMSLE
rmsle_cozy_corner = rmsle(y_test_cozy_corner, y_pred_cozy_corner)
print(f"RMSLE for 'Cozy Corner': {rmsle_cozy_corner}")


In [ ]:
import pandas as pd


data = {
    'Features': ['All Sales', 'Petalume','Cozy Corner','Candles','Accessories','Workshops','AromaGarden Gardenia candle','Candle Scent No. 8'],
    'RSMLE': ['0.13', '0.13','0.17','0.13','0.13','0.13','0.06','0.13']
}
RSMLE_per_feature = pd.DataFrame(data)


RSMLE_per_feature
#print(RSMLE_per_feature.to_string(index=False))

